In this example, I demonstrate a couple of useful things.  

I'm going to use Python because it is well-suited to this sort of stuff, but you may use whatever language with which you are most comfortable.
1. reading data from a CSV formatted file (valdosta.adsb)
2. reading data from a JSON formatted file (valdosta.gps)
3. merging these two files by datetime stamp, so that observations in each close in time are matched up.

About the data.  File valdosta.adsb contains summarized data of aircraft positions and altitudes. Here's what it looks like:

In [2]:
import pandas as pd
from pandas import read_csv

def main():
    adsb_df = read_csv( 'valdosta1.adsb', names=['date_time', 'mode', 'flight_id', 
        'hex_id', 'altitude', 'lat', 'lng', 'in_view'])
    print(adsb_df.head(20))

if __name__ == '__main__':
    main()


          date_time      mode flight_id  hex_id altitude       lat       lng  \
0          Starting       NaN       NaN     NaN      NaN       NaN       NaN   
1   20150618-133743  Updating     XXXXX  AA2A8A    40000   0.00000   0.00000   
2   20150618-133743  Updating     XXXXX  A60EDB    21000   0.00000   0.00000   
3   20150618-133743  Updating     XXXXX  AD6035    13775   0.00000   0.00000   
4   20150618-133743  Updating     XXXXX  A1AC69    13100   0.00000   0.00000   
5   20150618-133743  Updating     XXXXX  A29EB6    30450   0.00000   0.00000   
6   20150618-133743  Updating     XXXXX  A5E48A    36000   0.00000   0.00000   
7   20150618-133743  Updating     XXXXX  A2172E        0   0.00000   0.00000   
8   20150618-133743  Updating     XXXXX  AD18A3    28300   0.00000   0.00000   
9   20150618-133743  Updating     XXXXX  A3A0EB    36000   0.00000   0.00000   
10  20150618-133743  Updating     XXXXX  AC8CF1    36975   0.00000   0.00000   
11  20150618-133743  Updating     XXXXX 

I got the idea for how to do this from here:
https://stackoverflow.com/questions/15006298/how-to-preview-a-part-of-a-large-pandas-dataframe-in-ipython-notebook
Clearly the data still need some filtering but we're on the right track and pandas is doing a lot of the heavy lifting for us.

The json formatted file contains GPS position data for my aircraft receiver, which was located in a moving car.

Let's look at reading data from the corresponding json-formatted file:

In [4]:
import pandas as pd
from pandas import read_json

def main():
    gps_df = read_json( 'valdosta1.gps', lines=True)
    print(gps_df.head(50))

if __name__ == '__main__':
    main()

                   activated    alt     bps    class  climb  cycle  \
0                        NaN    NaN     NaN  VERSION    NaN    NaN   
1                        NaN    NaN     NaN  DEVICES    NaN    NaN   
2                        NaN    NaN     NaN    WATCH    NaN    NaN   
3   2015-06-18T03:08:59.900Z    NaN  9600.0   DEVICE    NaN    1.0   
4                        NaN  227.0     NaN      TPV    NaN    NaN   
5                        NaN  227.0     NaN      TPV    NaN    NaN   
6                        NaN    NaN     NaN      SKY    NaN    NaN   
7                        NaN  227.0     NaN      TPV    NaN    NaN   
8                        NaN  227.0     NaN      TPV    0.0    NaN   
9   2015-06-18T03:09:01.205Z    NaN  9600.0   DEVICE    NaN    1.0   
10  2015-06-18T03:09:01.926Z    NaN  9600.0   DEVICE    NaN    1.0   
11                       NaN  227.0     NaN      TPV    0.0    NaN   
12                       NaN  227.0     NaN      TPV    0.0    NaN   
13                  

I got this idea from: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html
Not all the data is shown in this abbreviated (and messy) preview.  Specifically, the latitude and longitude, two attributes of great interest here, are lost in the ellipsis (...).  Don't worry, we'll filter this data frame down in a subsequent step.

Let's go back to the CSV file and filter for just the rows we want:

In [5]:
import pandas as pd
from pandas import read_csv

def main():

    # Read the entire file
    adsb_df = read_csv( 'valdosta1.adsb', names=['date_time', 'mode', 'flight_id', 
        'hex_id', 'altitude', 'lat', 'lng', 'in_view'])

    # first subsetting expression -- only want updates, non-zero lat/long
    adsb_clean = adsb_df.loc[(adsb_df['mode'] == 'Updating') & (adsb_df['lat'] != 0 ) 
        & (adsb_df['lat'].notnull())]
        
    print(adsb_clean.head(20))

if __name__ == '__main__':
    main()


           date_time      mode flight_id  hex_id altitude       lat       lng  \
14   20150618-133746  Updating     XXXXX  A34721    27950  34.80688 -84.91154   
19   20150618-133752  Updating  784       A1D4C3    36050  34.32422 -85.04091   
32   20150618-133847  Updating     XXXXX  A34721    26500  34.95886 -85.11237   
37   20150618-133847  Updating     XXXXX  A399D4    27325  35.02758 -85.26049   
41   20150618-133853  Updating     XXXXX  AA7757    41000  33.78484 -83.94429   
43   20150618-133853  Updating  784       A1D4C3    36700  34.43559 -85.13917   
47   20150618-133912  Updating  UAL1096   AA7E2E    36000  34.83902 -84.98340   
51   20150618-133930  Updating     XXXXX  AB5F00    37000  34.14088 -85.70468   
60   20150618-133952  Updating     XXXXX  A34721    25925  34.99672 -85.16281   
65   20150618-133952  Updating     XXXXX  A399D4    27325  35.02758 -85.26049   
70   20150618-133955  Updating     XXXXX  AA7757    41000  33.78484 -83.94429   
72   20150618-133955  Updati

My code is based on this: https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/
and: https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values

Now let's clean the json data:

In [6]:
import pandas as pd
from pandas import read_json

def main():
    gps_df = read_json( 'valdosta1.gps', lines=True)

    # GPS data stream is very rich - a lot of redundant info, including 
    # info on the satellites themselves.  We only want position (fix) related
    # stuff.  TPV is Time, Position, Velocity
    gps_clean = gps_df.loc[gps_df['class'] == 'TPV']

    # Now keep only columns/fields we want
    gps_clean = gps_clean.filter( items=['time', 'lat', 'lon', 'speed', 'alt', 'track', 'climb'])

    # Deal with missing times
    gps_clean = gps_clean.loc[(gps_clean['time'].notnull()) & gps_clean['climb'].notnull()]
    print(gps_clean.head(20))

if __name__ == '__main__':
    main()

                        time      lat       lon  speed    alt   track  climb
8   2015-06-18T13:34:56.000Z  34.3793 -84.90703  0.010  227.0  140.06    0.0
11  2015-06-18T13:34:57.000Z  34.3793 -84.90703  0.010  227.0  136.46    0.0
12  2015-06-18T13:34:58.000Z  34.3793 -84.90703  0.010  227.0  134.16    0.0
13  2015-06-18T13:34:59.000Z  34.3793 -84.90703  0.005  227.0  244.59    0.0
15  2015-06-18T13:35:00.000Z  34.3793 -84.90703  0.005  227.0  288.26    0.0
16  2015-06-18T13:35:01.000Z  34.3793 -84.90703  0.010  227.0  318.41    0.0
17  2015-06-18T13:35:02.000Z  34.3793 -84.90703  0.010  227.0  332.62    0.0
18  2015-06-18T13:35:03.000Z  34.3793 -84.90703  0.010  226.9  326.85   -0.1
19  2015-06-18T13:35:04.000Z  34.3793 -84.90703  0.010  226.9  330.28    0.0
21  2015-06-18T13:35:05.000Z  34.3793 -84.90703  0.005  226.9   39.10    0.0
22  2015-06-18T13:35:06.000Z  34.3793 -84.90703  0.010  226.9  114.21    0.0
23  2015-06-18T13:35:07.000Z  34.3793 -84.90703  0.015  226.9  155.59    0.0

Ok, let's use Pandas to merge these files by timestamp.  The first step is to convert the string times into real Datetime type, starting with the plane data (from csv):

In [ ]:
import pandas as pd
from pandas import read_csv
from datetime import datetime

def main():

    # Read the entire file
    adsb_df = read_csv( 'valdosta1.adsb', names=['date_time', 'mode', 'flight_id', 
        'hex_id', 'altitude', 'lat', 'lng', 'in_view'])

    # first subsetting expression -- only want updates, non-zero lat/long
    adsb_clean = adsb_df.loc[(adsb_df['mode'] == 'Updating') & (adsb_df['lat'] != 0 ) 
        & (adsb_df['lat'].notnull())]

    # Convert string date to datetime internal format, add to dataframe as dt_tm
    # Ref: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
    # and: https://stackoverflow.com/questions/18942506/add-new-column-in-pandas-dataframe-python

    # kludge to turn off the dreaded SettingWithCopyWarning.  Discussion is  in SO link above.
    pd.options.mode.chained_assignment = None
    
    # Compute a new column with a datetime format from the string date time column
    adsb_clean['dt_tm'] = adsb_clean['date_time'].map( lambda x: datetime.strptime(x, '%Y%m%d-%H%M%S'))
    
    print(adsb_clean.head(20))

if __name__ == '__main__':
    main()


Now let's do the GPS data from the json file:

In [10]:
import pandas as pd
from pandas import read_json
from datetime import datetime

def main():
    gps_df = read_json( 'valdosta1.gps', lines=True)

    # GPS data stream is very rich - a lot of redundant info, including 
    # info on the satellites themselves.  We only want position (fix) related
    # stuff.  TPV is Time, Position, Velocity
    gps_clean = gps_df.loc[gps_df['class'] == 'TPV']

    # Now keep only columns/fields we want
    gps_clean = gps_clean.filter( items=['time', 'lat', 'lon', 'speed', 'alt', 'track', 'climb'])

    # Deal with missing times
    gps_clean = gps_clean.loc[(gps_clean['time'].notnull()) & gps_clean['climb'].notnull()]

    # Convert string date to datetime internal format, add to dataframe as dt_tm
    # Ref: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
    # and: https://stackoverflow.com/questions/18942506/add-new-column-in-pandas-dataframe-python

    # kludge to turn off the dreaded SettingWithCopyWarning.  Discussion is  in SO link above.
    pd.options.mode.chained_assignment = None

    # Compute a new column with a datetime format from the string date time column
    # string dates and times look like this 2015-06-18T13:43:40.000Z
    gps_clean['dt_tm'] = gps_clean['time'].map( lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000Z'))
    print(gps_clean.head(20))

if __name__ == '__main__':
    main()

                        time      lat       lon  speed    alt   track  climb  \
8   2015-06-18T13:34:56.000Z  34.3793 -84.90703  0.010  227.0  140.06    0.0   
11  2015-06-18T13:34:57.000Z  34.3793 -84.90703  0.010  227.0  136.46    0.0   
12  2015-06-18T13:34:58.000Z  34.3793 -84.90703  0.010  227.0  134.16    0.0   
13  2015-06-18T13:34:59.000Z  34.3793 -84.90703  0.005  227.0  244.59    0.0   
15  2015-06-18T13:35:00.000Z  34.3793 -84.90703  0.005  227.0  288.26    0.0   
16  2015-06-18T13:35:01.000Z  34.3793 -84.90703  0.010  227.0  318.41    0.0   
17  2015-06-18T13:35:02.000Z  34.3793 -84.90703  0.010  227.0  332.62    0.0   
18  2015-06-18T13:35:03.000Z  34.3793 -84.90703  0.010  226.9  326.85   -0.1   
19  2015-06-18T13:35:04.000Z  34.3793 -84.90703  0.010  226.9  330.28    0.0   
21  2015-06-18T13:35:05.000Z  34.3793 -84.90703  0.005  226.9   39.10    0.0   
22  2015-06-18T13:35:06.000Z  34.3793 -84.90703  0.010  226.9  114.21    0.0   
23  2015-06-18T13:35:07.000Z  34.3793 -8

Ok, let's merge using Pandas.  For simple merges, you could probably just round to the nearest minute and work that way, but I wanted to try the more elegant Pandas method:

In [11]:
import pandas as pd
from pandas import read_csv
from pandas import read_json
from datetime import datetime

def read_and_clean_json_file():
    gps_df = read_json( 'valdosta1.gps', lines=True)

    # GPS data stream is very rich - a lot of redundant info, including 
    # info on the satellites themselves.  We only want position (fix) related
    # stuff.  TPV is Time, Position, Velocity
    gps_clean = gps_df.loc[gps_df['class'] == 'TPV']

    # Now keep only columns/fields we want
    gps_clean = gps_clean.filter( items=['time', 'lat', 'lon', 'speed', 'alt', 'track', 'climb'])

    # Deal with missing times
    gps_clean = gps_clean.loc[(gps_clean['time'].notnull()) & gps_clean['climb'].notnull()]

    # Convert string date to datetime internal format, add to dataframe as dt_tm
    # Ref: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
    # and: https://stackoverflow.com/questions/18942506/add-new-column-in-pandas-dataframe-python

    # kludge to turn off the dreaded SettingWithCopyWarning.  Discussion is  in SO link above.
    pd.options.mode.chained_assignment = None

    # Compute a new column with a datetime format from the string date time column
    # string dates and times look like this 2015-06-18T13:43:40.000Z
    gps_clean['dt_tm'] = gps_clean['time'].map( lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000Z'))

    return gps_clean

def read_and_clean_csv_file():

    # Read the entire file
    adsb_df = read_csv( 'valdosta1.adsb', names=['date_time', 'mode', 'flight_id', 
        'hex_id', 'altitude', 'lat_plane', 'lng_plane', 'in_view'])

    # first subsetting expression -- only want updates, non-zero lat/long
    adsb_clean = adsb_df.loc[(adsb_df['mode'] == 'Updating') & (adsb_df['lat_plane'] != 0 ) 
        & (adsb_df['lat_plane'].notnull())]

    # Convert string date to datetime internal format, add to dataframe as dt_tm
    # Ref: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
    # and: https://stackoverflow.com/questions/18942506/add-new-column-in-pandas-dataframe-python

    # kludge to turn off the dreaded SettingWithCopyWarning.  Discussion is  in SO link above.
    pd.options.mode.chained_assignment = None

    # Compute a new column with a datetime format from the string date time column
    adsb_clean['dt_tm'] = adsb_clean['date_time'].map( lambda x: datetime.strptime(x, '%Y%m%d-%H%M%S'))

    return adsb_clean

# Based on: https://stackoverflow.com/questions/34880539/pandas-merging-based-on-a-timestamp-which-do-not-match-exactly
# also: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge_asof.html
def merge_planes_gps(planes, gps):
    tol = pd.Timedelta( '1 minute')
    all_df = pd.merge_asof(
        left=planes,
        right=gps,
        direction='nearest',
        on='dt_tm',
        tolerance=tol)
    return all_df

def print_selected( result, n_rows):
    print( result[['hex_id', 'date_time', 'lat_plane','lng_plane', 'time', 'lat', 'lon']].head(n_rows))

def main():

    adsb_clean = read_and_clean_csv_file( )
    gps_clean = read_and_clean_json_file( )

    result_df = merge_planes_gps( adsb_clean, gps_clean)
    print( 'Selected rows')
    print_selected( result_df, 50)

if __name__ == '__main__':
    main()


Selected rows
    hex_id        date_time  lat_plane  lng_plane                      time  \
0   A34721  20150618-133746   34.80688  -84.91154  2015-06-18T13:37:46.000Z   
1   A1D4C3  20150618-133752   34.32422  -85.04091  2015-06-18T13:37:52.000Z   
2   A34721  20150618-133847   34.95886  -85.11237  2015-06-18T13:38:47.000Z   
3   A399D4  20150618-133847   35.02758  -85.26049  2015-06-18T13:38:47.000Z   
4   AA7757  20150618-133853   33.78484  -83.94429  2015-06-18T13:38:53.000Z   
5   A1D4C3  20150618-133853   34.43559  -85.13917  2015-06-18T13:38:53.000Z   
6   AA7E2E  20150618-133912   34.83902  -84.98340  2015-06-18T13:39:12.000Z   
7   AB5F00  20150618-133930   34.14088  -85.70468  2015-06-18T13:39:30.000Z   
8   A34721  20150618-133952   34.99672  -85.16281  2015-06-18T13:39:52.000Z   
9   A399D4  20150618-133952   35.02758  -85.26049  2015-06-18T13:39:52.000Z   
10  AA7757  20150618-133955   33.78484  -83.94429  2015-06-18T13:39:55.000Z   
11  A1D4C3  20150618-133955   34.52174